In [67]:
#!/usr/bin python3
import requests
from Crypto.Util.number import bytes_to_long, getPrime
import Crypto.Random.random as random
from hashlib import sha256
from gmpy2 import gcd, invert
from json import loads

session = requests.Session()
print(session.cookies.get_dict())

response = session.get('http://192.168.16.128:8000/public_key')
print(response.text)

{}
{"n":"19419268746348857757491853239358917595573235919745788178589792784099026741230155576090864953184156304490935653268122006025026832998925631039893878550717180487605338015658226748375778390806103261079356416871461298446176246140076922321020405769116068330353323040450771358414261564428912861331573275147258347619345924676696191120722431208064939637753728612564541089862698357949546225804212298975446744134162298036702875709003570295957077328108817839185754463936333656493747648838832586760638070313901445638633739598283106886483648931701198188781411457987866384332374075328944076330281065949640935244955484305029721180887","e":"65537"}


In [68]:
pubkey = loads(response.text)
N = int(pubkey["n"])
e = int(pubkey["e"])
print(N, e)

19419268746348857757491853239358917595573235919745788178589792784099026741230155576090864953184156304490935653268122006025026832998925631039893878550717180487605338015658226748375778390806103261079356416871461298446176246140076922321020405769116068330353323040450771358414261564428912861331573275147258347619345924676696191120722431208064939637753728612564541089862698357949546225804212298975446744134162298036702875709003570295957077328108817839185754463936333656493747648838832586760638070313901445638633739598283106886483648931701198188781411457987866384332374075328944076330281065949640935244955484305029721180887 65537


In [70]:
response = session.post('http://192.168.16.128:8000/api/auth/register', json={"username": "1t111es12112t1t","password": "1testt"})
print(response.text)

In [71]:
a = b"2020-11-28"
m = b"test message"
r = N
while gcd(r, N) != 1 :
    r = random.randint(1, N)
r_p = N
while gcd(r_p, N) != 1 :
    r_p = random.randint(1, N)
u = N
while gcd(u, N) != 1 :
    u = random.randint(1, N)
alpha = (pow(((r**3)*r_p),e, N) # (r^3r')^e
         * bytes_to_long(sha256(m).digest()) #h(m) 
         * (u**2 + 1)) % N

In [72]:
response = session.post('http://192.168.16.128:8000/api/auth/pre_sign', json={"a": "2020-11-28", "alpha": str(alpha)})
print(response.text)

{"x":"8549935033918125349722391844227715221108108397232946800807838901274462274605378986224957687301961471405447618647855135106715542667928830081288921513223814867591781905786234943094330238613697128620277915848352516907131871067050755799109989261653160826951041472950788151948820468492704447960491263850566420629960220255748568193150196388860125705302895141280006140139698941537563936522784963054400226230273703177276952244659194245680224456177202811160106013552497400215063410185677067155680592233740621472908610606437150148549538928124357510588489954796052237864669202530728535037636027689068909643811129201229954539287"}


In [73]:
x = int(loads(response.text)["x"])
print(x)

8549935033918125349722391844227715221108108397232946800807838901274462274605378986224957687301961471405447618647855135106715542667928830081288921513223814867591781905786234943094330238613697128620277915848352516907131871067050755799109989261653160826951041472950788151948820468492704447960491263850566420629960220255748568193150196388860125705302895141280006140139698941537563936522784963054400226230273703177276952244659194245680224456177202811160106013552497400215063410185677067155680592233740621472908610606437150148549538928124357510588489954796052237864669202530728535037636027689068909643811129201229954539287


In [74]:
beta = (pow(r, e, N) * (u - x)) % N
response = session.post('http://192.168.16.128:8000/api/auth/sign', json={"a": "2020-11-28", "alpha": str(alpha), "beta": str(beta)})
print(response.text)

{"beta_invert":"12354608355440834463191979282528222345188470621854187831734022455356815362008096098652538200955670624921582646366041598197528265746167602331122677653003489072161955470889577555172043929036222616438875710768311221406366742407020355578518605090040393377058079362011703931765788068551010358956378985608300254728569207225686738944081815120462417080878811783625301910766685197980732945977370254361910906255599143846011972398611001914834560763586863830299802245440178376928355535554523706548405045573759020329962711318006481379551768562864970218369963422308839840883945307273749636493263335835711152783548482722561689434451","t":"987624764987210054616377436068127178439991433424331617533716802960700490372286726088922829395835976442053084465295930843499670396711779685623748893850080892658934264970802364918751788481201486719442723642014752307481348123918848514694561446357217859681091672514202757302800158933929862306549868391118949134650167887597484178886995392081761494713969211256949703

In [75]:
sig = loads(response.text)
beta_ = int(sig["beta_invert"])
T = int(sig["t"])
print(beta_, T)

12354608355440834463191979282528222345188470621854187831734022455356815362008096098652538200955670624921582646366041598197528265746167602331122677653003489072161955470889577555172043929036222616438875710768311221406366742407020355578518605090040393377058079362011703931765788068551010358956378985608300254728569207225686738944081815120462417080878811783625301910766685197980732945977370254361910906255599143846011972398611001914834560763586863830299802245440178376928355535554523706548405045573759020329962711318006481379551768562864970218369963422308839840883945307273749636493263335835711152783548482722561689434451 9876247649872100546163774360681271784399914334243316175337168029607004903722867260889228293958359764420530844652959308434996703967117796856237488938500808926589342649708023649187517884812014867194427236420147523074813481239188485146945614463572178596810916725142027573028001589339298623065498683911189491346501678875974841788869953920817614947139692112569497032102449372528261400642

In [76]:
c = ((u * x + 1) * invert(u-x, N)) % N
s = (T 
      * bytes_to_long(sha256(a).digest()) 
      * bytes_to_long(sha256(m).digest())
      * bytes_to_long(sha256(m).digest()) 
      * pow(r*r_p, 2 * e - 2, N) 
      * ((c*c + 1) ** 2)) % N
print(c)
print(s)

17479120057154425962247104659799621816658552786820395115023368369753179089631330317716260229292981960920611666436197950172218624700927428597509659558746476205199830814832061099038739302669214676692284551548755534157537633989079661437001770523429453160325367006240889927686707330301350950010648471582886228635133162154023443734249484003530903643901469726923598519971525459499533518615956464269559121310533909452497601091669353726359565321051858717268429023577796748744058446558693943837387665178482838405642620354584153460773798943865910338462320879931362660389848418299548465559342590793929221048637933713680096002762
4910573408356210160347374071791093722606016798165475165365372957328342913808283309298816455661378855666347303367165311181969232546655716767961845051238612538771245491612412430708606090251872685014362971449444899106509325260570552939841550406709226675292899974889541370842221167465882739178506690033685940099885027671008929463849612840049355495818888894989933583001671058107861980536

In [77]:
response = session.post('http://192.168.16.128:8000/verify', json={"a": "2020-11-28","c": str(c), "s": str(s), "m": "test message"})
print(response)
print(response.text)

<Response [200]>
Signed Message


In [78]:
print("s^e={}".format(pow(s, e, N)))
print("h(a)h(m)^2(c^2+1)^2={}".format(
    bytes_to_long(sha256(a).digest())
    * (bytes_to_long(sha256(m).digest()) ** 2)
    * ((c ** 2 + 1) ** 2)
    % N))

s^e=13084057822672627808179938343506906991648634045750380623869882641715023739605275472009446701621575194337899811741937556635036369741711424862354716023994386147139596365553458017475124889509188732808649395466012503109603060092263596413225303997871818827206779882623034384758963607783471445225266964272512453229225954904069054015876306304730235687458763303127564317422030656522926419482460691569334559209821990252767896765344950543128860079964975449570635402890595046672720011431332388663802291812928730589022242628022920852634817105243445014356420512818076816188308898824884559492905894539315240495745547928093885821098
h(a)h(m)^2(c^2+1)^2=1308405782267262780817993834350690699164863404575038062386988264171502373960527547200944670162157519433789981174193755663503636974171142486235471602399438614713959636555345801747512488950918873280864939546601250310960306009226359641322530399787181882720677988262303438475896360778347144522526696427251245322922595490406905401587630630473023568745876330312756